<img src='https://ikomia.com/wp-content/uploads/2022/10/banner.png'>

# How to train YOLO v7 with Ikomia API

Object detection is an important task in Computer Vision. Lots of companies use this technology:


*   **Smart city** : crowd analysis or infrastructure inspection
*   **Retail** : inventory tracking or customer tracking
*   **Autonomous vehicles** : vehicle or pedestrian detection
*   **Security** : facial recognition or potential threats detection
*   **Manufacturing** : quality control or defect detection
*   **Healthcare** : many disease recognition

If you are familiar with object detection, you know that there exists a lot of algorithms and the latest, **YOLO v7** is one of the most **powerful** at this time of writing.

I will not describe the algorithm because you can already find some good articles on the web. You just need to know that YOLO v7 is very fast (real time processing) and accurate. 

The main bottleneck for many developers is...how can I use this technology in my own problem ?

In this tutorial, I show you how it can be EASY to create your own object detector thanks to Ikomia API.

Paper : [YOLOv7: Trainable bag-of-freebies sets new state-of-the-art for real-time object detectors](https://arxiv.org/abs/2207.02696)

Code : [GitHub repository](https://github.com/wongkinyiu/yolov7)


ENJOY 🥰 !!

<p float="left">
  <img src="https://ikomia.com/wp-content/uploads/2022/10/img_foot.png" width="500" /> 
    <img src="https://ikomia.com/wp-content/uploads/2022/10/img_bbox.png" width="500" />
</p>

## Setup

Please use a GPU for this tutorial.

In the menu, select "Runtime" then "Change runtime type", choose GPU in "Hardware accelerator".

Check your GPU with the following command:

In [ ]:
!nvidia-smi

First of all, you need to install Ikomia API pip package.

In [ ]:
!pip install ikomia



---


** -Google Colab ONLY- Restart runtime**

Click on the "RESTART RUNTIME" button at the end the previous window.





---

Use your credentials to connect with Ikomia HUB and access more than 60 algorithms and 800+ models.

Your credentials are the same as you use to login to our website, if you don't have any credentials, follow this link

https://members.ikomia.com/accounts/signup/

In [ ]:
import ikomia
import os

# Easy and unsafe authentication | Only for personal use
os.environ['IKOMIA_USER'] = "your_login"
os.environ['IKOMIA_PWD'] = "your_password"

ikomia.authenticate()

## How to train YOLO v7 on a custom YOLO dataset

Download your dataset from your preferred tool. In this example, we use a dataset from **Roboflow** which is a great annotation platform used by many developers and companies. The dataset is exported in YOLO format.

In [ ]:
!curl -L "https://universe.roboflow.com/ds/997v9b3SK6?key=qTrYR6x8fv" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip
!cp $(pwd)/valid/* $(pwd)/train/

In order to train YOLOv7 on your custom dataset, please create a new workflow from scratch.

Then you need 2 components:

1.   A YOLO dataset loader which loads dataset in YOLO format and convert it to an Ikomia format
2.   The YOLOv7 training algorithm which loads dataset in Ikomia format

Add these 2 previous algorithms to your workflow and then it will automagically download all algorithms from Ikomia Hub and install all the Python dependencies (the 1st time, it can take a while, be patient ! ).

** -Google Colab ONLY- ** If you want to monitor your training results, you can use TensorBoard by running the following cell.

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /root/Ikomia/Tensorboard

Now, it's time to train your model !

In [ ]:
from ikomia.utils import ik
from ikomia.core import task
from ikomia.dataprocess import workflow
import os

#----------------------------- Step 1 -----------------------------------#
# Create a workflow which will take your dataset as input and
# train a YOLOv7 model on it
#------------------------------------------------------------------------#
wf = workflow.create("Train YOLOv7")

#----------------------------- Step 2 -----------------------------------#
# First you need to convert darknet YOLO format to IKOMIA format.
# Add an Ikomia dataset converter to your workflow.
#------------------------------------------------------------------------#
dataset_yolo_id, dataset_yolo = wf.add_task(ik.dataset_yolo)

# Specify where are the data and class file
dataset_yolo_params = {
    ik.dataset_yolo_param.data_folder_path: os.getcwd()+"/train",
    ik.dataset_yolo_param.class_file_path: os.getcwd()+"/train/_darknet.labels"
}
wf.set_parameters(task_id=dataset_yolo_id, params=dataset_yolo_params)

#----------------------------- Step 3 -----------------------------------#
# Then, you want to train a YOLOv7 model.
# Add YOLOv7 training algorithm to your workflow
#------------------------------------------------------------------------#
yolo7_id, yolo7 = wf.add_task(ik.train_yolo_v7)

# Specify where to put the pre-trained model and set hyperparameters
train_yolo_v7_params = {
    ik.train_yolo_v7_param.output_folder: os.getcwd(),
    ik.train_yolo_v7_param.batch_size: 16, # <-- Reduce the batch size if you encounter some "Cuda out of memory"
    ik.train_yolo_v7_param.epochs: 150
}
wf.set_parameters(task_id=yolo7_id, params=train_yolo_v7_params)


#----------------------------- Step 4 -----------------------------------#
# Connect your tasks I/O (Input/Output).
# In this example, all connections are automatic because "dataset_yolo"
# outputs exactly fit with "train_yolo" inputs
#------------------------------------------------------------------------#
wf.connect_tasks(dataset_yolo_id, yolo7_id)
#----------------------------- Step 5 -----------------------------------#
# Execute your workflow.
# It automatically runs all your tasks sequentially.
#------------------------------------------------------------------------#
wf.run()

## How to execute YOLOv7 on images

Once the training is finished, you may want to experiment the fresh model on new test images. Just use the following code to execute YOLO v7 inference on your image.

In [ ]:
from ikomia.dataprocess import workflow, registry
from ikomia.utils import ik
import cv2

# Create your workflow for YOLO inference
wf = workflow.create("YOLOv7 inference")

# Add YOLO v7 to your workflow
yolo7_id, yolo7 = wf.add_task(ik.infer_yolo_v7)

# Set custom parameters
# Comment these 4 lines if you want to use pretrained model on COCO
yolo7_params = {
    ik.infer_yolo_v7_param.custom_train: True,
    ik.infer_yolo_v7_param.custom_model: os.getcwd()+"/folder_name/weights/best.pt", # <-- replace "folder_name" by the correct folder name in /content/
    ik.infer_yolo_v7_param.thr_conf: 0.25
}
wf.set_parameters(task_id=yolo7_id, params=yolo7_params)

# Connect YOLO v7 with the root task
wf.connect_tasks(wf.getRootID(), yolo7_id)

# Apply YOLO v7 on your image
wf.run_on(path=os.getcwd()+"/train/yt1s-com---Resume-Tottenham-20-West-Bromwich-Premier-League-J23_720p-fps-2_00044_jpeg_jpg.rf.9de8afe0132987f478d16d536e768a67.jpg")

## Display your results

Utility function for displaying image

In [ ]:
def imshow(img):
    import cv2
    import IPython
    _,ret = cv2.imencode('.jpg', img) 
    i = IPython.display.Image(data=ret)
    IPython.display.display(i)

In [ ]:
# Get YOLO v7 image result
img_bbox = wf.get_image_with_graphics(task_id=yolo7_id)

# Display in Colab
img_bbox = cv2.cvtColor(img_bbox, cv2.COLOR_BGR2RGB)
imshow(img_bbox)

## -Google Colab ONLY- Save your YOLO v7 model in your Google Drive account



In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')

# Uncomment and insert the correct "folder name" in the following path
#%cp /content/foler_name/weights/best.pt /content/gdrive/MyDrive

## -Google Colab ONLY- Download directly your custom model

In [ ]:
#from google.colab import files

# Uncomment and insert the correct "folder name" in the following path
#files.download('/content/folder_name/weights/best.pt')